# Final Capstone: Revisiting the Netflix Prize

## Notebook 3: Correlation and Separation of Training and Quiz Sets

In [1]:
import time
start_time = time.perf_counter()
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)

In [2]:
%%time
# import main df and quiz index list
base_path = 'C:/Users/jnpol/Documents/DS/Data Science/UL/'
quindex = pd.read_parquet(base_path + 'quindex.parquet')
quilist = list(quindex.quindex)
net = pd.read_parquet(base_path + 'net2.parquet')
net.info()
net.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97713135 entries, 61342875 to 6598294
Data columns (total 24 columns):
 #   Column           Dtype  
---  ------           -----  
 0   mov_id           int16  
 1   cust_id          int32  
 2   rating           float64
 3   day_rated        int16  
 4   mov_year         int16  
 5   mov_count        int32  
 6   rated_bycust     int16  
 7   rate_each_day    int32  
 8   mov_day_count    uint16 
 9   cust_day_count   int16  
 10  cust_days_since  int16  
 11  mov_days_since   int16  
 12  mov_avg_rating   float32
 13  cust_avg_rating  float32
 14  mov_day_avg_rl   float32
 15  mov_day_avg      float32
 16  cust_avg_offset  float32
 17  cust_day_avg     float32
 18  avg_rate_mov_yr  float32
 19  avg_rate_cst_yr  float32
 20  global_mean      float32
 21  cust_glob_diff   float32
 22  mov_glob_diff    float32
 23  all_ratings      int8   
dtypes: float32(11), float64(1), int16(7), int32(3), int8(1), uint16(1)
memory usage: 8.1 GB
Wall t

,mov_id,cust_id,rating,day_rated,mov_year,mov_count,rated_bycust,rate_each_day,mov_day_count,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg_rl,mov_day_avg,cust_avg_offset,cust_day_avg,avg_rate_mov_yr,avg_rate_cst_yr,global_mean,cust_glob_diff,mov_glob_diff,all_ratings
61342875,11182,549302,4.00,1696,2003,103062,459,70730,296,106,4,278,3.55,3.79,3.53,3.53,0.19,3.97,3.51,3.75,3.60,-0.18,0.06,4
12081331,2342,1077245,3.00,1976,2004,87480,139,98810,126,3,0,420,3.87,4.27,3.72,3.72,0.47,3.67,3.52,3.94,3.60,-0.67,-0.26,3
99011741,17474,2463816,4.00,1567,1964,32089,1577,83753,41,7,125,525,4.09,3.08,4.20,4.20,-0.48,3.71,3.92,4.07,3.60,0.52,-0.49,4
73636568,13372,2204684,3.00,1225,2002,350,829,15570,1,5,546,191,2.69,3.26,3.00,3.00,-0.28,3.40,3.50,3.32,3.60,0.35,0.91,3
66797623,12191,2032601,3.00,1679,1996,12782,1219,97725,34,6,890,1612,2.97,3.48,2.65,2.65,0.04,3.67,3.55,3.34,3.60,0.12,0.63,3


## Correlation Matrix

Note: Correlation between a continuous variable and a constant is undefined, and will return 'nan' values.

PCA will be done on highly correlated variables in another notebook once the data is separated into features and target, then split.

In [3]:
%%time
net.corr()

Wall time: 1min 51s


,mov_id,cust_id,rating,day_rated,mov_year,mov_count,rated_bycust,rate_each_day,mov_day_count,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg_rl,mov_day_avg,cust_avg_offset,cust_day_avg,avg_rate_mov_yr,avg_rate_cst_yr,global_mean,cust_glob_diff,mov_glob_diff,all_ratings
mov_id,1.00,0.00,0.01,0.00,-0.00,0.04,-0.01,0.00,0.03,-0.00,-0.01,0.04,0.02,0.00,0.02,0.02,0.00,0.00,0.01,0.01,nan,-0.00,-0.02,0.01
cust_id,0.00,1.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,nan,0.00,0.00,-0.00
rating,0.01,-0.00,1.00,0.09,-0.08,0.10,-0.10,0.06,0.03,-0.03,-0.03,0.08,0.36,0.40,0.48,0.48,0.39,0.61,0.09,0.54,nan,-0.40,-0.36,1.00
day_rated,0.00,0.00,0.09,1.00,0.05,0.04,-0.13,0.59,0.05,0.07,0.09,0.33,0.10,0.19,0.19,0.19,0.13,0.15,-0.05,0.15,nan,-0.19,-0.10,0.09
mov_year,-0.00,-0.00,-0.08,0.05,1.00,0.27,-0.11,0.03,0.06,-0.10,0.02,-0.32,-0.22,0.02,-0.17,-0.17,0.05,-0.03,-0.88,-0.15,nan,-0.02,0.22,-0.08
mov_count,0.04,-0.00,0.10,0.04,0.27,1.00,-0.22,0.05,0.16,-0.07,-0.11,0.05,0.28,0.09,0.21,0.21,0.06,0.09,-0.26,0.04,nan,-0.09,-0.28,0.10
rated_bycust,-0.01,0.00,-0.10,-0.13,-0.11,-0.22,1.00,-0.09,-0.04,0.33,0.10,-0.04,-0.10,-0.25,-0.13,-0.13,-0.17,-0.17,0.11,-0.19,nan,0.25,0.10,-0.10
rate_each_day,0.00,0.00,0.06,0.59,0.03,0.05,-0.09,1.00,0.10,0.06,0.04,0.20,0.08,0.12,0.13,0.13,0.08,0.10,-0.03,0.10,nan,-0.12,-0.08,0.06
mov_day_count,0.03,-0.00,0.03,0.05,0.06,0.16,-0.04,0.10,1.00,-0.05,0.06,-0.06,0.10,0.01,0.07,0.07,0.00,0.04,-0.04,0.00,nan,-0.01,-0.10,0.03
cust_day_count,-0.00,0.00,-0.03,0.07,-0.10,-0.07,0.33,0.06,-0.05,1.00,-0.18,0.17,-0.01,-0.05,-0.01,-0.01,-0.04,-0.05,0.11,-0.04,nan,0.05,0.01,-0.03


In [4]:
%%time
# create quiz set from primary df by selecting rows with null values
quiz = net[net.isna().any(axis=1)].copy()

# drop the column 'rating' since it is all nans
quiz.drop(['rating'], 1, inplace=True)

# rename the column that contains the ratings to 'rating'
quiz.rename(columns={'all_ratings': 'rating'}, inplace=True)
quiz.info()
quiz.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408395 entries, 8994179 to 49941263
Data columns (total 23 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   mov_id           1408395 non-null  int16  
 1   cust_id          1408395 non-null  int32  
 2   day_rated        1408395 non-null  int16  
 3   mov_year         1408395 non-null  int16  
 4   mov_count        1408395 non-null  int32  
 5   rated_bycust     1408395 non-null  int16  
 6   rate_each_day    1408395 non-null  int32  
 7   mov_day_count    1408395 non-null  uint16 
 8   cust_day_count   1408395 non-null  int16  
 9   cust_days_since  1408395 non-null  int16  
 10  mov_days_since   1408395 non-null  int16  
 11  mov_avg_rating   1408395 non-null  float32
 12  cust_avg_rating  1408395 non-null  float32
 13  mov_day_avg_rl   1408395 non-null  float32
 14  mov_day_avg      1408395 non-null  float32
 15  cust_avg_offset  1408395 non-null  float32
 16  cust_day_av

,mov_id,cust_id,day_rated,mov_year,mov_count,rated_bycust,rate_each_day,mov_day_count,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg_rl,mov_day_avg,cust_avg_offset,cust_day_avg,avg_rate_mov_yr,avg_rate_cst_yr,global_mean,cust_glob_diff,mov_glob_diff,rating
8994179,1798,32439,2044,1987,110574,76,146052,163,76,0,2043,3.96,3.91,4.05,4.05,0.27,3.91,3.65,3.33,3.60,-0.30,-0.36,5
88877859,15792,1114824,1710,2003,804,17,72225,2,1,359,445,3.75,3.25,4.00,4.00,-0.46,3.62,3.51,3.25,3.60,0.35,-0.15,4
97938651,17324,1939417,2147,2005,106716,78,177424,356,2,270,271,3.88,3.86,3.64,3.64,0.24,2.00,3.55,2.00,3.60,-0.26,-0.28,4
3085518,571,2162086,2151,1999,150848,50,135400,208,37,227,1993,3.96,3.48,4.01,4.01,-0.30,3.58,3.60,4.75,3.60,0.13,-0.36,3
24041226,4493,433805,1921,2003,6215,449,161409,43,4,1745,212,3.14,3.29,3.00,3.00,-0.15,4.50,3.51,3.00,3.60,0.31,0.47,2


In [5]:
%%time
# split quiz set into features and target
quiz_target = quiz.drop(['mov_id', 'cust_id', 'day_rated', 'mov_year',
                         'mov_count', 'rated_bycust', 'rate_each_day',
                         'mov_day_count', 'cust_day_count', 'cust_days_since',
                         'mov_days_since', 'mov_avg_rating', 'cust_avg_rating',
                         'mov_day_avg_rl', 'mov_day_avg', 'cust_avg_offset',
                         'cust_day_avg', 'avg_rate_mov_yr', 'avg_rate_cst_yr',
                         'global_mean', 'cust_glob_diff', 'mov_glob_diff'], 1)
quiz_features = quiz.drop(['rating'], 1)
quiz_target.info()
display(quiz_target.head())
quiz_features.info()
quiz_features.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408395 entries, 8994179 to 49941263
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   rating  1408395 non-null  int8 
dtypes: int8(1)
memory usage: 12.1 MB


,rating
8994179,5
88877859,4
97938651,4
3085518,3
24041226,2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408395 entries, 8994179 to 49941263
Data columns (total 22 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   mov_id           1408395 non-null  int16  
 1   cust_id          1408395 non-null  int32  
 2   day_rated        1408395 non-null  int16  
 3   mov_year         1408395 non-null  int16  
 4   mov_count        1408395 non-null  int32  
 5   rated_bycust     1408395 non-null  int16  
 6   rate_each_day    1408395 non-null  int32  
 7   mov_day_count    1408395 non-null  uint16 
 8   cust_day_count   1408395 non-null  int16  
 9   cust_days_since  1408395 non-null  int16  
 10  mov_days_since   1408395 non-null  int16  
 11  mov_avg_rating   1408395 non-null  float32
 12  cust_avg_rating  1408395 non-null  float32
 13  mov_day_avg_rl   1408395 non-null  float32
 14  mov_day_avg      1408395 non-null  float32
 15  cust_avg_offset  1408395 non-null  float32
 16  cust_day_av

,mov_id,cust_id,day_rated,mov_year,mov_count,rated_bycust,rate_each_day,mov_day_count,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg_rl,mov_day_avg,cust_avg_offset,cust_day_avg,avg_rate_mov_yr,avg_rate_cst_yr,global_mean,cust_glob_diff,mov_glob_diff
8994179,1798,32439,2044,1987,110574,76,146052,163,76,0,2043,3.96,3.91,4.05,4.05,0.27,3.91,3.65,3.33,3.60,-0.30,-0.36
88877859,15792,1114824,1710,2003,804,17,72225,2,1,359,445,3.75,3.25,4.00,4.00,-0.46,3.62,3.51,3.25,3.60,0.35,-0.15
97938651,17324,1939417,2147,2005,106716,78,177424,356,2,270,271,3.88,3.86,3.64,3.64,0.24,2.00,3.55,2.00,3.60,-0.26,-0.28
3085518,571,2162086,2151,1999,150848,50,135400,208,37,227,1993,3.96,3.48,4.01,4.01,-0.30,3.58,3.60,4.75,3.60,0.13,-0.36
24041226,4493,433805,1921,2003,6215,449,161409,43,4,1745,212,3.14,3.29,3.00,3.00,-0.15,4.50,3.51,3.00,3.60,0.31,0.47


In [6]:
%%time
# drop the column 'rating' since it is not needed (and float64)
net.drop(['rating'], 1, inplace=True)

# rename the column that contains the ratings to 'rating'
net.rename(columns={'all_ratings': 'rating'}, inplace=True)

# drop the matching rows that belong only in the quiz set
net.drop(quilist, inplace=True)
net.info()
net.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96304740 entries, 61342875 to 6598294
Data columns (total 23 columns):
 #   Column           Dtype  
---  ------           -----  
 0   mov_id           int16  
 1   cust_id          int32  
 2   day_rated        int16  
 3   mov_year         int16  
 4   mov_count        int32  
 5   rated_bycust     int16  
 6   rate_each_day    int32  
 7   mov_day_count    uint16 
 8   cust_day_count   int16  
 9   cust_days_since  int16  
 10  mov_days_since   int16  
 11  mov_avg_rating   float32
 12  cust_avg_rating  float32
 13  mov_day_avg_rl   float32
 14  mov_day_avg      float32
 15  cust_avg_offset  float32
 16  cust_day_avg     float32
 17  avg_rate_mov_yr  float32
 18  avg_rate_cst_yr  float32
 19  global_mean      float32
 20  cust_glob_diff   float32
 21  mov_glob_diff    float32
 22  rating           int8   
dtypes: float32(11), int16(7), int32(3), int8(1), uint16(1)
memory usage: 7.3 GB
Wall time: 28.6 s


,mov_id,cust_id,day_rated,mov_year,mov_count,rated_bycust,rate_each_day,mov_day_count,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg_rl,mov_day_avg,cust_avg_offset,cust_day_avg,avg_rate_mov_yr,avg_rate_cst_yr,global_mean,cust_glob_diff,mov_glob_diff,rating
61342875,11182,549302,1696,2003,103062,459,70730,296,106,4,278,3.55,3.79,3.53,3.53,0.19,3.97,3.51,3.75,3.60,-0.18,0.06,4
12081331,2342,1077245,1976,2004,87480,139,98810,126,3,0,420,3.87,4.27,3.72,3.72,0.47,3.67,3.52,3.94,3.60,-0.67,-0.26,3
99011741,17474,2463816,1567,1964,32089,1577,83753,41,7,125,525,4.09,3.08,4.20,4.20,-0.48,3.71,3.92,4.07,3.60,0.52,-0.49,4
73636568,13372,2204684,1225,2002,350,829,15570,1,5,546,191,2.69,3.26,3.00,3.00,-0.28,3.40,3.50,3.32,3.60,0.35,0.91,3
66797623,12191,2032601,1679,1996,12782,1219,97725,34,6,890,1612,2.97,3.48,2.65,2.65,0.04,3.67,3.55,3.34,3.60,0.12,0.63,3


In [7]:
%%time
# split training set into features and target
train_target = net.drop(['mov_id', 'cust_id', 'day_rated', 'mov_year',
                         'mov_count', 'rated_bycust', 'rate_each_day',
                         'mov_day_count', 'cust_day_count', 'cust_days_since',
                         'mov_days_since', 'mov_avg_rating', 'cust_avg_rating',
                         'mov_day_avg_rl', 'mov_day_avg', 'cust_avg_offset',
                         'cust_day_avg', 'avg_rate_mov_yr', 'avg_rate_cst_yr',
                         'global_mean', 'cust_glob_diff', 'mov_glob_diff'], 1)
train_features = net.drop(['rating'], 1)
train_target.info()
print()
train_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96304740 entries, 61342875 to 6598294
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   rating  int8 
dtypes: int8(1)
memory usage: 826.6 MB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96304740 entries, 61342875 to 6598294
Data columns (total 22 columns):
 #   Column           Dtype  
---  ------           -----  
 0   mov_id           int16  
 1   cust_id          int32  
 2   day_rated        int16  
 3   mov_year         int16  
 4   mov_count        int32  
 5   rated_bycust     int16  
 6   rate_each_day    int32  
 7   mov_day_count    uint16 
 8   cust_day_count   int16  
 9   cust_days_since  int16  
 10  mov_days_since   int16  
 11  mov_avg_rating   float32
 12  cust_avg_rating  float32
 13  mov_day_avg_rl   float32
 14  mov_day_avg      float32
 15  cust_avg_offset  float32
 16  cust_day_avg     float32
 17  avg_rate_mov_yr  float32
 18  avg_rate_cst_yr  float32
 19  global_mean      float32
 2

In [8]:
%%time
quiz_target.to_parquet('quiz_target.parquet')
del quiz_target
quiz_features.to_parquet('quiz_features.parquet')
del quiz_features
train_target.to_parquet('train_target.parquet')
del train_target
train_features.to_parquet('train_features.parquet')

Wall time: 37.7 s


In [8]:
end_time = time.perf_counter()
prog_ex_time = end_time - start_time
print('This program executes in {} seconds.'.format(prog_ex_time))

This program executes in 69.5177171 seconds.


That's 1 minute and 9.52 seconds.